<a href="https://colab.research.google.com/github/nitron-alpha-kplr/Machine-Learning/blob/main/prediction_diabetes_pyspark_mllib_corrige.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TACHE 1 : Installation des dépendances et exécution de la session Spark

In [ ]:
#installer PySpark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 8.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=8584f602e964ecc76cafafc03e7a7eeda49f7d63d75d1c7ab9afcfc0a0d34570
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
#créer une session Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark").getOrCreate()

# TACHE 2: Cloner et explorer le jeu de données

In [ ]:
#cloner le jeu de données sur le diabète depuis le dépôt github
!git clone https://github.com/education454/diabetes_dataset

Cloning into 'diabetes_dataset'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 13.00 KiB | 578.00 KiB/s, done.


In [ ]:
#vérifier si le jeu de données existe
!ls diabetes_dataset

diabetes.csv  new_test.csv


In [ ]:
#créer un dataframe spark
df = spark.read.csv('/content/diabetes_dataset/diabetes.csv', header=True, inferSchema=True)

In [ ]:
#afficher le dataframe
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|      0|25.6|                   0.294| 28|      0|


In [ ]:
#afficher le schéma
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [ ]:
#afficher le nombre de lignes et le nombre de colonnes du jeu de données
print((df.count(), len(df.columns)))

(2000, 9)


In [ ]:
#compter le nombre total de diabétiques et de non-diabétiques
df.groupby('Outcome').count().show()

+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  684|
|      0| 1316|
+-------+-----+



In [ ]:
#obtenir les statistiques sommaires
df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

# TACHE 3: Nettoyage et préparation des données

In [ ]:
#vérification des valeurs nulles
for col in df.columns:
  print(col+ ":", df[df[col].isNull()].count())

Pregnancies: 0
Glucose: 0
BloodPressure: 0
SkinThickness: 0
Insulin: 0
BMI: 0
DiabetesPedigreeFunction: 0
Age: 0
Outcome: 0


In [ ]:
#rechercher les valeurs inutiles présentes
def count_zeros():
  column_list= ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
  for i in column_list:
    print(i+ ":", df[df[i] == 0].count())

In [ ]:
count_zeros()

Glucose: 13
BloodPressure: 90
SkinThickness: 573
Insulin: 956
BMI: 28


In [ ]:
#calculer et remplacer les valeurs inutiles par la valeur moyenne
from pyspark.sql.functions import *
for i in df.columns[1:6]:
  data = df.agg({i:'mean'}).first()[0]
  print("Mean value for {} is {}".format(i,int(data)))
  df = df.withColumn(i, when(df[i]== 0, int(data)).otherwise(df[i]))

Mean value for Glucose is 121
Mean value for BloodPressure is 69
Mean value for SkinThickness is 20
Mean value for Insulin is 80
Mean value for BMI is 32


In [ ]:
#afficher le dataframe
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|     80|25.6|                   0.294| 28|      0|


# TACHE 4: Analyse de corrélation et sélection des caractéristiques

In [ ]:
#trouver la corrélation entre l'ensemble des variables d'entrée et de sortie
for col in df.columns:
  print("Corelation to outcome for {} is {}".format(col, df.stat.corr('Outcome',col)))

Corelation to outcome for Pregnancies is 0.22443699263363961
Corelation to outcome for Glucose is 0.48796646527321064
Corelation to outcome for BloodPressure is 0.17171333286446713
Corelation to outcome for SkinThickness is 0.1659010662889893
Corelation to outcome for Insulin is 0.1711763270226193
Corelation to outcome for BMI is 0.2827927569760082
Corelation to outcome for DiabetesPedigreeFunction is 0.1554590791569403
Corelation to outcome for Age is 0.23650924717620253
Corelation to outcome for Outcome is 1.0


In [ ]:
#sélection des caractéristiques
#nous choisirons toutes les caractéristiques d'entrée de notre modèle comme étant des co_relation similaires
from pyspark.ml.feature import VectorAssembler
#définition d'une nouvelle colonne de caractéristiques
assembler = VectorAssembler(inputCols=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'], outputCol='features')
output_data = assembler.transform(df) # produce a new column 'features

In [ ]:
#afficher le schéma
output_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
#afficher le dataframe
output_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|[2.0,138.0,62.0,3...|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[0.0,84.0,82.0,31...|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|[0.0,145.0,69.0,2...|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|[0.0,135.0,68.0,4...|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|[1.0,139.0,62.0,4...|
|          0|    173|           

# TACHE 5: Diviser l'ensemble de données et construire le modèle

In [ ]:
#create final data
#nous allons maintenant utiliser le modèle de l'algorithme de régression logistique
from pyspark.ml.classification import LogisticRegression
# Sélection des caractéristiques et de la colonne des résultats
final_data = output_data.select('features', 'Outcome')

In [ ]:
#afficher le schéma de final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Outcome: integer (nullable = true)



In [ ]:
#diviser l'ensemble de données ; construire le modèle
train, test = final_data.randomSplit([0.7,0.3])

In [ ]:
#entraîner le modèle
models = LogisticRegression(labelCol='Outcome')
model = models.fit(train)

In [ ]:
#résumé du modèle
summary = model.summary

In [ ]:
summary.predictions.describe().show()

+-------+------------------+------------------+
|summary|           Outcome|        prediction|
+-------+------------------+------------------+
|  count|              1406|              1406|
|   mean|0.3385490753911807|0.2581792318634424|
| stddev|0.4733846029259937|0.4377887974791848|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



# TACHE 6: Évaluer et sauvegarder le modèle

In [ ]:
# Par défaut, il utilise l'aire sous le ROC comme matrice de performance.
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model.evaluate(test)

In [ ]:
predictions.predictions.show(10)

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[0.0,57.0,60.0,20...|      0|[4.30196969637327...|[0.98663907242784...|       0.0|
|[0.0,67.0,76.0,20...|      0|[2.67971584965431...|[0.93581905914180...|       0.0|
|[0.0,67.0,76.0,20...|      0|[2.67971584965431...|[0.93581905914180...|       0.0|
|[0.0,73.0,69.0,20...|      0|[4.29723593298395...|[0.98657652603571...|       0.0|
|[0.0,74.0,52.0,10...|      0|[3.63965128759247...|[0.97441051810085...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.56182378755123...|[0.92836384246427...|       0.0|
|[0.0,84.0,64.0,22...|      0|[2.56182378755123...|[0.92836384246427...|       0.0|
|[0.0,86.0,68.0,32...|      0|[2.83338254486400...|[0.94445332159701...|       0.0|
|[0.0,91.0,68.0,32...|      0|[2.38697662224463...|[0.91582879958255...|    

In [ ]:
#en utilisant l'évaluateur de classification binaire
evaluator =BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='Outcome')
# Quelle est la justesse (accuracy) de notre modèle ?
evaluator.evaluate(model.transform(test))

0.8264746911119958

In [ ]:
#sauvegarder le modèle
model.save("model")

In [ ]:
#load saved model back to the environment
from pyspark.ml.classification import LogisticRegressionModel
model = LogisticRegressionModel.load('model')

# TACHE 7: Prédiction sur de nouvelles données avec le modèle sauvegardé

In [ ]:
#créer un nouveau dataframe Spark
test_df = spark.read.csv("/content/diabetes_dataset/new_test.csv", header=True, inferSchema=True)

In [ ]:
#afficher le schéma
test_df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)



In [ ]:
#créer une colonne fusionnée supplémentaire
test_data = assembler.transform(test_df)

In [ ]:
#afficher le schéma
test_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
#utiliser le modèle pour faire des prédictions
results = model.transform(test_data)
results.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [ ]:
#afficher les prévisions
results.select('features', 'prediction').show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[1.0,190.0,78.0,3...|       1.0|
|[0.0,80.0,84.0,36...|       0.0|
|[2.0,138.0,82.0,4...|       1.0|
|[1.0,110.0,63.0,4...|       1.0|
+--------------------+----------+

